In [3]:
#NYT API docs: https://developer.nytimes.com/apis
import requests
import json
import pandas as pd
import time
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob

In [17]:
#all articles from one month (Jan 2019)
r = requests.get('https://api.nytimes.com/svc/archive/v1/2019/1.json?api-key=hUqxCPIjU7aBGGXMK1pfw7wK5LKtgLjS')

In [32]:
#all articles containing "climate change" from 2020
climate = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json?q=climate+change&begin_date=20200101&end_date=20201231&api-key=hUqxCPIjU7aBGGXMK1pfw7wK5LKtgLjS')

In [60]:
#one article
r.json()['response']['docs'][2]
#number of articles returned 
len(r.json()['response']['docs'])

4482

In [19]:
#article title
r.json()['response']['docs'][1]['headline']['main']

KeyError: 'ma'

In [17]:
climate_docs = climate.json()['response']['docs']

NameError: name 'climate' is not defined

In [30]:
climate_docs[1]['lead_paragraph']

'The very ice melt he warned would threaten the planet turned out to be his undoing.'

In [62]:
len(climate_docs)
type(climate_docs)

list

In [3]:
#function to insert page number, date range in url; date format YYYYMMDD
#also input search words & your api key
def make_search_url(pg_num, start_date, end_date):
    url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?q=climate+change&begin_date=" + str(start_date) + "&end_date=" + str(end_date) + "&page=" + str(pg_num) + "&api-key=hUqxCPIjU7aBGGXMK1pfw7wK5LKtgLjS"
    return url

In [6]:
#Note: this will take a while to run 
climate_docs = []
for i in range(0,10):
    url = make_search_url(i,20200101,20200530)
    req = requests.get(url)
    all_docs = req.json()['response']['docs']
    climate_docs = climate_docs + all_docs
    print(i)
    print(url)
    print(len(climate_docs))
    if (len(all_docs) < 10):
        break 
    time.sleep(7) #pause 7 seconds

0
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=climate+change&begin_date=20200101&end_date=20200530&page=0&api-key=hUqxCPIjU7aBGGXMK1pfw7wK5LKtgLjS
10


KeyboardInterrupt: 

In [25]:
#initialize empty lists that will be columns of data frame
title = []
date = []
news_desk = []
abstract = []
url = []

In [26]:
#iterate through docs and extract data
for doc in climate_docs:
    title.append(doc['headline']['main'])
    date.append(doc['pub_date'])
    news_desk.append(doc['news_desk'])
    abstract.append(doc['abstract'])
    url.append(doc['web_url'])

In [12]:
#make dictionary & convert to pandas dataframe & save as csv
df = {'title': title,
     'date': date,
     'news_desk': news_desk,
     'abstract': abstract,
     'url' : url}
df = pd.DataFrame(df) 
df.to_csv('NYT_climate.csv')

In [13]:
#import csv as pandas dataframe
articles = pd.read_csv('NYT_climate.csv')

In [19]:
def make_comments_url(article_url):
    url = "https://api.nytimes.com/svc/community/v3/user-content/url.json?api-key=hUqxCPIjU7aBGGXMK1pfw7wK5LKtgLjS&offset=0&url=" + article_url
    return url

'https://www.nytimes.com/2020/05/18/climate/climate-changes-hurricane-intensity.html'

In [15]:
#all comments from one article
c = requests.get("https://api.nytimes.com/svc/community/v3/user-content/url.json?api-key=hUqxCPIjU7aBGGXMK1pfw7wK5LKtgLjS&offset=0&url=https%3A%2F%2Fwww.nytimes.com%2F2019%2F06%2F21%2Fscience%2Fgiant-squid-cephalopod-video.html")

In [35]:
c = requests.get("https://api.nytimes.com/svc/community/v3/user-content/url.json?api-key=hUqxCPIjU7aBGGXMK1pfw7wK5LKtgLjS&offset=0&url=https%3A%2F%2Fwww.nytimes.com/2020/05/27/opinion/michigan-edenville-dam.html")

In [27]:
url

['https://www.nytimes.com/2020/05/27/climate/nyt-climate-newsletter-pop-songs.html',
 'https://www.nytimes.com/2020/05/18/climate/climate-changes-hurricane-intensity.html',
 'https://www.nytimes.com/2020/05/29/climate/coronavirus-economic-stimulus-climate.html',
 'https://www.nytimes.com/2020/05/21/climate/dam-failure-michigan-climate-change.html',
 'https://www.nytimes.com/2020/05/07/climate/coronavirus-republicans-climate-change.html',
 'https://www.nytimes.com/2020/05/26/climate/cop26-climate-talks.html',
 'https://www.nytimes.com/interactive/2020/05/21/climate/songs-about-climate-change.html',
 'https://www.nytimes.com/2020/05/23/climate/climate-change-coronavirus.html',
 'https://www.nytimes.com/2020/05/19/climate/coronavirus-climate-change-survey.html',
 'https://www.nytimes.com/2020/05/27/opinion/michigan-edenville-dam.html']

In [36]:
c.json()

{'debug': {'version': 4, 'nyt-com-shim': True},
 'status': 'OK',
 'copyright': 'Copyright (c) 2021 The New York Times Company.  All Rights Reserved.',
 'results': {'callerID': None,
  'api_timestamp': '1609620526',
  'comments': [],
  'depthLimit': 2,
  'filter': '',
  'page': 0,
  'replyLimit': 3,
  'sort': '',
  'totalCommentsFound': 0,
  'totalCommentsReturned': 0,
  'totalEditorsSelectionFound': 0,
  'totalEditorsSelectionReturned': 0,
  'totalParentCommentsFound': 0,
  'totalParentCommentsReturned': 0,
  'totalRecommendationsFound': 0,
  'totalRecommendationsReturned': 0,
  'totalReplyCommentsFound': 0,
  'totalReplyCommentsReturned': 0,
  'totalReporterReplyCommentsFound': 0,
  'totalReporterReplyCommentsReturned': 0,
  'userData': {'userID': 0,
   'login': None,
   'email': '',
   'displayName': '',
   'url': '',
   'title': '',
   'location': '',
   'times_people': 0,
   'sharing': 0,
   'labels': []}},
 'errorDetails': 'Comments are not enabled on this article'}